In [1]:
### INITIALIZE VARIABLES ###
project_name = "ee-hungletatdac12345" # Name of the project on your google cloud project
drive_folder= "Thanh Hoa Satellite Images 2" # Name of the folder in your google drive where the data is stored
numRectsX = 5  # Number of rectangles along the X axis
numRectsY = 5 # Number of rectangles along the Y axis


In [3]:

import ee
# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project=project_name)

# Define the polygon around Canberra, Australia large enough for 8 rectangles

hoang_hoa_poly = [
    [105.7595, 19.9614],
    [105.9666, 19.9614],
    [105.9666, 19.7699],
    [105.7595, 19.7699],
    [105.7595, 19.9614]
]
quan_son_poly=[
    [104.6063, 20.4116],
    [105.1309, 20.4116],
    [105.1309, 20.0953],
    [104.6063, 20.0953],
    [104.6063, 20.4116]
]

hoang_hoa_polygon = ee.Geometry.Polygon(hoang_hoa_poly)
quan_son_polygon = ee.Geometry.Polygon(quan_son_poly)
# Parameters for the number of rectangles

def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Function to subdivide the polygon into rectangles based on the number of rectangles
def subdivide_polygon(polygon, numRectsX, numRectsY):
    bounds = polygon.bounds()
    coords = bounds.coordinates().get(0).getInfo()
    xMin = coords[0][0]
    xMax = coords[2][0]
    yMin = coords[0][1]
    yMax = coords[2][1]

    rectWidth = (xMax - xMin) / numRectsX
    rectHeight = (yMax - yMin) / numRectsY

    rectangles = []

    for x in range(numRectsX):
        for y in range(numRectsY):
            x0 = xMin + x * rectWidth
            y0 = yMin + y * rectHeight
            x1 = x0 + rectWidth
            y1 = y0 + rectHeight
            rect = {
                "rectangle":ee.Geometry.Rectangle([x0, y0, x1, y1]),
                "position": (x, y)
            }
            rectangles.append(rect)

    return rectangles

# Function to export images to Google Drive
def prepare_export_tasks(place_name: str, rectangles, from_date: str, to_date: str ):
    tasks = []
    for index, rectangle in enumerate(rectangles):
        rect=rectangle['rectangle']
        position=rectangle['position']
        image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(rect) \
            .filterDate(from_date, to_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                
        image=image.map(maskS2clouds)\
            .median() \
            .select(['B4', 'B3', 'B2', 'B8', 'B11']) \
            .clip(rect)

        task = ee.batch.Export.image.toDrive(
            image=image,
            description = f"{place_name}_MONTH_{from_date.split('-')[1]}_{str(position[0])}_{str(position[1])}_{str(index)}",
            folder=drive_folder,
            scale=5,  # Adjust this scale as needed
            region=rect,
            maxPixels=1e9,
            fileFormat='GeoTIFF',
            formatOptions={'cloudOptimized': True}
        )
        tasks.append(task)
    
    return tasks

# Call the function to prepare the tasks
hoang_hoa_rectangles = subdivide_polygon(hoang_hoa_polygon, numRectsX, numRectsY)
quan_son_rectangles = subdivide_polygon(quan_son_polygon, numRectsX, numRectsY)


tasks=[]
for month in range(1, 13):
    if month == 12:
        hoanghoa_temp = prepare_export_tasks('HoangHoa', hoang_hoa_rectangles, f'2023-{month:02d}-01', f'2023-12-31')
        quanson_temp = prepare_export_tasks('QuanSon', quan_son_rectangles, f'2023-{month:02d}-01', f'2023-12-31')
    else:
        hoanghoa_temp = prepare_export_tasks('HoangHoa', hoang_hoa_rectangles, f'2023-{month:02d}-01', f'2023-{month+1:02d}-01')
        quanson_temp = prepare_export_tasks('QuanSon', quan_son_rectangles, f'2023-{month:02d}-01', f'2023-{month+1:02d}-01')
    
    temp = hoanghoa_temp + quanson_temp
    tasks.extend(temp)

# tasks=prepare_export_tasks('HoangHoa', [hoang_hoa_rectangles[0]], '2023-03-01', '2023-04-01')
# tasks=prepare_export_tasks('QuanSon', quan_son_rectangles, '2023-09-01', '2023-10-01')
# Start the tasks
for task in tasks:
    task.start()
    print(f"Exporting {task.config['description']}")


Exporting QuanSon_MONTH_09_0_0_0
Exporting QuanSon_MONTH_09_0_1_1
Exporting QuanSon_MONTH_09_0_2_2
Exporting QuanSon_MONTH_09_0_3_3
Exporting QuanSon_MONTH_09_0_4_4
Exporting QuanSon_MONTH_09_1_0_5
Exporting QuanSon_MONTH_09_1_1_6
Exporting QuanSon_MONTH_09_1_2_7
Exporting QuanSon_MONTH_09_1_3_8
Exporting QuanSon_MONTH_09_1_4_9
Exporting QuanSon_MONTH_09_2_0_10
Exporting QuanSon_MONTH_09_2_1_11
Exporting QuanSon_MONTH_09_2_2_12
Exporting QuanSon_MONTH_09_2_3_13
Exporting QuanSon_MONTH_09_2_4_14
Exporting QuanSon_MONTH_09_3_0_15
Exporting QuanSon_MONTH_09_3_1_16
Exporting QuanSon_MONTH_09_3_2_17
Exporting QuanSon_MONTH_09_3_3_18
Exporting QuanSon_MONTH_09_3_4_19
Exporting QuanSon_MONTH_09_4_0_20
Exporting QuanSon_MONTH_09_4_1_21
Exporting QuanSon_MONTH_09_4_2_22
Exporting QuanSon_MONTH_09_4_3_23
Exporting QuanSon_MONTH_09_4_4_24


In [6]:
hoang_hoa_rectangles[0]

{'rectangle': ee.Geometry({
   "functionInvocationValue": {
     "functionName": "GeometryConstructors.Polygon",
     "arguments": {
       "coordinates": {
         "constantValue": [
           [
             [
               105.7595,
               19.80820600505867
             ],
             [
               105.7595,
               19.76989999999998
             ],
             [
               105.80092,
               19.76989999999998
             ],
             [
               105.80092,
               19.80820600505867
             ]
           ]
         ]
       },
       "evenOdd": {
         "constantValue": true
       }
     }
   }
 }),
 'position': (0, 0)}